In [1]:
import pandas as pd
import pyodbc

DB = {'servername': r'LAPTOP-NDOACQTH\SQLEXPRESS',
      'database': 'Practicum_4'}

# Constructing the connection string with Windows Authentication
conn_str = (
    'Driver={SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'Trusted_Connection=yes;'
)

# Establishing the connection
export_conn = pyodbc.connect(conn_str)
export_cursor = export_conn.cursor()
export_cursor

In [3]:
inventory_levels_df = pd.read_csv('GO_SALES_INVENTORY_LEVELSData.csv')
inventory_levels_df

,INVENTORY_YEAR,INVENTORY_MONTH,PRODUCT_NUMBER,INVENTORY_COUNT
2021,4,48,1932,NaN
2021,4,49,1400,NaN
2021,4,50,21705,NaN
2021,4,51,9710,NaN
2021,4,52,5616,NaN
...,...,...,...,...
2022,12,111,1128,NaN
2022,12,112,7810,NaN
2022,12,113,3485,NaN
2022,12,114,350,NaN


In [7]:
inventory_levels_df = inventory_levels_df.astype(str)

cursor = export_conn.cursor()

# Itereer over de rijen van het DataFrame en voeg deze toe aan de database
for index, row in inventory_levels_df.iterrows():
    cursor.execute("INSERT INTO [SALES_INVENTORY_LEVELS] (INVENTORY_YEAR, INVENTORY_MONTH, PRODUCT_NUMBER, INVENTORY_COUNT) VALUES (?, ?, ?, ?)",
                   row['INVENTORY_YEAR'], row['INVENTORY_MONTH'], row['PRODUCT_NUMBER'], row['INVENTORY_COUNT'])

# Transactie commiten
export_conn.commit()

# Cursor en verbinding sluiten
cursor.close()
export_conn.close()

IntegrityError: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_SALES_INVENTORY_LEVELS'. Cannot insert duplicate key in object 'dbo.SALES_INVENTORY_LEVELS'. The duplicate key value is (4, 48). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")

In [4]:
product_forecast_df = pd.read_csv('GO_SALES_PRODUCT_FORECASTData.csv')
product_forecast_df

,PRODUCT_NUMBER,YEAR,MONTH,EXPECTED_VOLUME
0,44,2022,12,383
1,45,2021,1,80
2,45,2021,2,51
3,45,2021,3,214
4,45,2021,4,300
...,...,...,...,...
3867,115,2022,8,282
3868,115,2022,9,920
3869,115,2022,10,1081
3870,115,2022,11,398
